### Diff for bike flows (time evolution)

Given two months, this notebook computes what has changed on the most *relevant* flows from one month to another:
* which new relevant flows have appeared (plotted in green)
* which old relevant flows are not relevant anymore (plotted in red)

Here, **relevant** means *flows belonging to the 4th tier in that month*.

In [1]:
import glob
from ipywidgets import interact_manual, widgets
from IPython.core.display import display, HTML

import bikescience.grid as grid
import bikescience.load_trips as tr
import bikescience.flow_diffs as diffs
import bikescience.stations as st

/home/usuario/.local/lib/python3.8/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/home/usuario/Documents/bss_ic/bss-analysis/routes/bikescience/grid.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readt

In [2]:
def plot_diffs(start, start_period, end, end_period, grid_size, perc_below):
    print('Calculating...')
    boston_grid = grid.Grid(n=grid_size)
    start_filter = tr.period_functions[start_period]
    end_filter = tr.period_functions[end_period]
    start_trips = start_filter(tr.load_trips_file(start))
    print('start_trips')
    display(start_trips)
    end_trips = end_filter(tr.load_trips_file(end))
    # display(start_trips)
    fmap = diffs.diffs_map(start_trips, end_trips, boston_grid, stations, stations_distances, perc_below)
    print('Done.')
    file = 'maps/diffs.html'
    fmap.save(file)
    display(HTML('Saved at <a href="' + file + '" target="_blank">' + file + '</a>'))
    display(fmap)

In [3]:
import pandas as pd
trips = pd.read_csv('../data/trips/loaded_trips/trips_few.csv')
trips

,date,tripduration,start_station_name_old,start_station_id,starttime,end_station_name_old,end_station_id,stoptime,birth_year,age,...,week_day,weekend,holiday,start_station_name,end_station_name,lat_start,lon_start,lat_end,lon_end,distance
0,2022-01-01,889,85 - Praça do Ciclista,85.0,2022-01-01 11:00:46,187 - Parque Buenos Aires,187,2022-01-01 11:15:35,1998-01-01 00:00:00,24.0,...,5,True,False,Praça do Ciclista,Parque Buenos Aires,-23.555958,-46.662666,-23.545607,-46.657056,1.555
1,2022-01-01,2071,85 - Praça do Ciclista,85.0,2022-01-01 12:02:31,44 - Ibirapuera Portão 09,44,2022-01-01 12:37:02,1993-01-01 00:00:00,29.0,...,5,True,False,Praça do Ciclista,Pq. Ibirapuera portão 9,-23.555958,-46.662666,-23.578840,-46.660090,3.616
2,2022-01-01,679,70 - Alameda Casa Branca,70.0,2022-01-01 11:19:51,44 - Ibirapuera Portão 09,44,2022-01-01 11:31:10,2002-01-01 00:00:00,20.0,...,5,True,False,Alameda Casa Branca,Pq. Ibirapuera portão 9,-23.569903,-46.661879,-23.578840,-46.660090,1.446
3,2022-01-01,1148,44 - Ibirapuera Portão 09,44.0,2022-01-01 15:06:20,86 - Shopping Center 3,86,2022-01-01 15:25:28,1988-01-01 00:00:00,34.0,...,5,True,False,Pq. Ibirapuera portão 9,Shopping Center 3,-23.578840,-46.660090,-23.558425,-46.659610,3.193
4,2022-01-01,539,91 - Paulista 778,91.0,2022-01-01 14:08:48,86 - Shopping Center 3,86,2022-01-01 14:17:47,1966-01-01 00:00:00,56.0,...,5,True,False,Paulista 778,Shopping Center 3,-23.566250,-46.650436,-23.558425,-46.659610,1.856
5,2022-01-01,14364,86 - Shopping Center 3,86.0,2022-01-01 20:11:46,86 - Shopping Center 3,86,2022-01-02 00:11:10,2003-01-01 00:00:00,19.0,...,5,True,False,Shopping Center 3,Shopping Center 3,-23.558425,-46.659610,-23.558425,-46.659610,0.000
6,2022-01-01,1270,307 - Metrô Vila Mariana,307.0,2022-01-01 14:52:13,86 - Shopping Center 3,86,2022-01-01 15:13:23,1988-01-01 00:00:00,34.0,...,5,True,False,Metrô Vila Mariana,Shopping Center 3,-23.589802,-46.635085,-23.558425,-46.659610,5.467
7,2022-01-01,2429,146 - Ibirapuera portão 03,146.0,2022-01-01 16:34:03,146 - Ibirapuera portão 03,146,2022-01-01 17:14:32,2002-01-01 00:00:00,20.0,...,5,True,False,Ibirapuera Portão 03,Ibirapuera Portão 03,-23.589043,-46.652787,-23.589043,-46.652787,0.000
8,2022-01-01,647,130 - Afonso Braz,130.0,2022-01-01 12:32:24,130 - Afonso Braz,130,2022-01-01 12:43:11,1999-01-01 00:00:00,23.0,...,5,True,False,Afonso Braz,Afonso Braz,-23.595892,-46.664178,-23.595892,-46.664178,0.000
9,2022-01-01,1182,322 - Metrô Eucaliptos,322.0,2022-01-01 14:51:12,312 - Parque da Bicicletas,312,2022-01-01 15:10:54,1995-01-01 00:00:00,27.0,...,5,True,False,Metrô Eucaliptos,Parque das Bicicletas,-23.609453,-46.668144,-23.600556,-46.657754,1.616


In [4]:
!pip install rtree
!pip install pygeos

In [5]:
# folder = '../data/boston/bike-trips/'
# files = list(map(lambda f: (f.replace(folder, '')[:6], f), glob.glob(folder + '*')))
# files.sort()
# files = list(map(lambda f: (f[0][4:6] + '/' + f[0][:4], f[1]), files))

data_folder = '../data/'

files = list([data_folder + 'trips/loaded_trips/trips_few.csv'])

stations = st.load(data_folder + 'stations/treated_data/2021.11.08_Endereços-BikeSAMPA.csv')
stations_distances = st.distances(data_folder + 'stations/distance/stations_distance.csv')

im = interact_manual(
    plot_diffs, 
    start=widgets.Dropdown(options=files), 
    start_period=widgets.Dropdown(options=[('entire period', 0), ('morning', 1), ('lunchtime', 2), ('afternoon', 3)]),
    end=widgets.Dropdown(options=files),
    end_period=widgets.Dropdown(options=[('entire period', 0), ('morning', 1), ('lunchtime', 2), ('afternoon', 3)]),
    grid_size=widgets.SelectionSlider(options=[10, 20, 30, 40], value=20),
    perc_below=widgets.IntSlider(min=0, max=100, value=20, step=1)
)
im.widget.children[6].description = 'Plot Differences'

interactive(children=(Dropdown(description='start', options=('../data/trips/loaded_trips/trips_few.csv',), val…

In [6]:
files

['../data/trips/loaded_trips/trips_few.csv']